# Notebook `04_model_deep_learning.ipynb`

---

# 🧠 Construction et Entraînement du Modèle Deep Learning

Ce notebook constitue la **quatrième étape du pipeline IA**.  
Il est dédié à la construction, l’entraînement, l’évaluation et la sauvegarde d’un **modèle de deep learning** destiné à prédire le **prix d’un service Fiverr** à partir de variables numériques et vectorielles.

## 🎯 Objectifs

- Charger les données propres et transformées (`fiverr_cleaned_transformed.csv`)
- Générer les **embeddings vectoriels** à partir des descriptions (modèle `SentenceTransformer`)
- Préparer les **entrées combinées** : texte vectorisé, niveau encodé, fiabilité numérique
- Définir un **modèle Keras séquentiel** adapté à la régression
- Réaliser une **séparation train/test** et standardiser les variables
- Entraîner le modèle avec **early stopping** pour éviter le surapprentissage
- Évaluer ses performances sur les données de test
- Sauvegarder le modèle (`deep_model.h5`) et le scaler (`scaler.pkl`)

## ✅ Compétences mobilisées

- **Bloc 3 — C3** : Implémenter un modèle de deep learning adapté à un jeu de données structuré
- **Bloc 3 — C2** : Préparer les données et normaliser les vecteurs d’entrée (embedding + features classiques)
- **Bloc 5 — C4** : Exporter un modèle exploitable dans un environnement déployé (API, Gradio)

*Ce notebook prépare un modèle de prédiction avancé basé sur les réseaux de neurones, utilisé dans l’application finale.*

---

## 🧭 Sommaire

1. [Importation des bibliothèques pour le Deep Learning](#-1-importation-des-bibliothèques-pour-le-deep-learning)
2. [Chargement des données transformées dans un DataFrame Pandas](#-2-chargement-des-données-transformées-dans-un-dataframe-pandas)
3. [Préparation des données pour le Deep Learning](#-3-préparation-des-données-pour-le-deep-learning)
4. [Construction du modèle MLP (régression du prix)](#-4-construction-du-modèle-mlp-régression-du-prix)
5. [Entraînement du modèle avec EarlyStopping](#-5-entraînement-du-modèle-avec-earlystopping)
6. [Évaluation du modèle entraîné](#-6-évaluation-du-modèle-entraîné)
7. [Sauvegarde du modèle Deep Learning entraîné](#-7-sauvegarde-du-modèle-deep-learning-entraîné)


---

## 🧠 1. Importation des bibliothèques pour le Deep Learning

### ❓ 1.1. Pourquoi cette étape maintenant ?

Avant toute manipulation ou entraînement, nous devons importer toutes les **bibliothèques nécessaires** à la gestion des données, à la construction du modèle, et à l’ingénierie des variables.

Cela garantit :
- un environnement prêt à exécuter le pipeline complet,
- une meilleure lisibilité du script,
- et la centralisation des dépendances en début de fichier.

### 🎯 1.2. Résultat attendu

- Toutes les librairies utiles au traitement et à l'entraînement d’un modèle Deep Learning sont importées.
- L'importation est **clairement organisée** par type de tâche (données, modèle, I/O, etc.).
- Aucune erreur d'importation ne bloque l'exécution du notebook.

---

### 🐍 1.3. Script d’importation des bibliothèques nécessaires

In [8]:
# Importation des bibliothèques nécessaires

# Manipulation de données
import pandas as pd   # Bibliothèque pour la gestion des tableaux de données (DataFrame)
import numpy as np    # Bibliothèque pour le calcul numérique performant (vecteurs, matrices, etc.)

# Deep Learning avec TensorFlow Keras
import tensorflow as tf  # Backend TensorFlow (nécessaire même si Keras est utilisé seul)
from tensorflow.keras.models import Sequential       # Modèle linéaire empilé (séquentiel)
from tensorflow.keras.layers import Dense, Dropout   # Couches dense (fully connected) et dropout (régularisation)
from tensorflow.keras.callbacks import EarlyStopping # Callback pour arrêter l'entraînement en cas de surapprentissage

# Préparation et évaluation des données
from sklearn.model_selection import train_test_split  # Fonction de séparation du dataset en ensembles d'entraînement/test
from sklearn.preprocessing import StandardScaler       # Standardisation (centrage/réduction) des variables numériques

# Gestion des fichiers et modèles
import os       # Outils de gestion de fichiers et répertoires
import joblib   # Sauvegarde et chargement efficace des objets Python (modèles, scalers, etc.)

# Embedding de texte via transformers
from sentence_transformers import SentenceTransformer  # Génération d’embeddings vectoriels à partir de textes

---

## 📂 2. Chargement des données transformées dans un DataFrame Pandas

### ❓ 2.1. Pourquoi cette étape maintenant ?

Le fichier `fiverr_cleaned_transformed.csv` contient les données **nettoyées et enrichies** suite aux étapes de prétraitement précédentes.  
C’est à partir de ce fichier que nous allons **préparer les entrées** du modèle de deep learning.

Cette étape permet :
- de **valider l’accès au fichier** et le bon format CSV,
- d’initialiser le DataFrame `df` pour les traitements ultérieurs,
- d’obtenir une **confirmation immédiate** sur le nombre de lignes et colonnes disponibles.

### 🎯 2.2. Résultat attendu

- Les données sont correctement lues dans le DataFrame `df`.
- Aucune erreur d’accès ou de lecture n’est rencontrée.
- Le terminal affiche les dimensions des données (nombre de lignes et de colonnes).

---

### 🐍 2.3. Script de chargement des données transformées

In [9]:
# 🔹 Chargement des données

# Définition du chemin vers le fichier CSV nettoyé et transformé
file_path = "../data/fiverr_cleaned_dl_notebook.csv"

# Chargement du fichier CSV dans un DataFrame Pandas
df = pd.read_csv(file_path)

# Affichage d'un message de confirmation avec les dimensions des données chargées
print("Données chargées :", df.shape)  # Exemple : (25000, 10)

Données chargées : (1145, 5)


---

## 🧪 3. Préparation des données pour le Deep Learning

### ❓ 3.1. Pourquoi cette étape maintenant ?

Cette étape prépare les **entrées du modèle de deep learning** :
- Génération des **embeddings vectoriels** pour les descriptions textuelles,
- **Encodage one-hot** de la variable catégorielle `Niveau`,
- Sélection et concaténation des **variables numériques** comme `Fiabilite`,
- Séparation du jeu de données en ensembles d’entraînement et de test,
- **Normalisation des features** pour stabiliser l’apprentissage,
- Sauvegarde du scaler pour reproduire le pipeline d’inférence plus tard.

C’est une étape centrale avant toute modélisation supervisée.

### 🎯 3.2. Résultat attendu

- Le DataFrame `X` contient toutes les variables explicatives correctement formatées.
- Les jeux `X_train_scaled`, `X_test_scaled`, `y_train`, `y_test` sont prêts à l’usage.
- Le scaler `StandardScaler` est sauvegardé dans `models/deep/scaler.pkl`.
- Les dimensions du jeu d’entraînement sont affichées à l’écran pour validation.

---

### 🐍 3.3. Script de préparation des features d'entrée pour le deep learning

In [10]:
# Chargement des données
df = pd.read_csv(file_path)  # Lecture du fichier transformé contenant les features prêtes à l’emploi

# Chargement du modèle d'embedding SentenceTransformer
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # Modèle léger et performant pour vectoriser les descriptions textuelles

# Embedding de la colonne 'Description'
descriptions = df["Description"].astype(str).tolist()
embeddings = embedding_model.encode(descriptions)
embed_df = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(embeddings.shape[1])])  # Création d’un DataFrame pour les vecteurs d’embedding

# Encodage one-hot du niveau du vendeur
niveau_encoded = pd.get_dummies(df["Niveau"], prefix="Niveau")  # Conversion de la variable catégorielle en variables binaires

# Sélection des variables numériques restantes
autres_features = df[["Fiabilite"]].reset_index(drop=True)  # Ajout de la variable numérique "Fiabilite"

# Fusion finale des features dans X
X = pd.concat([embed_df, niveau_encoded, autres_features], axis=1)  # Construction du tableau final de variables explicatives
y = df["Prix"]  # Variable cible : le prix

# Découpage du jeu de données en train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80% entraînement, 20% test

# Standardisation des données (centrage-réduction)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Apprentissage des paramètres sur X_train
X_test_scaled = scaler.transform(X_test)        # Transformation de X_test avec les mêmes paramètres

# Sauvegarde du scaler pour une réutilisation future
os.makedirs("../models/deep", exist_ok=True)
scaler_path = "../models/deep/scaler_notebook.pkl"
joblib.dump(scaler, scaler_path)

# Messages de vérification
print("Scaler sauvegardé :", scaler_path)
print("Données prêtes pour entraînement deep learning :", X_train_scaled.shape)

Scaler sauvegardé : ../models/deep/scaler_notebook.pkl
Données prêtes pour entraînement deep learning : (916, 388)


---

## 🧠 4. Construction du modèle MLP (régression du prix)

### ❓ 4.1. Pourquoi cette étape maintenant ?

Nous allons entraîner un modèle de Deep Learning de type **MLP (Multilayer Perceptron)** pour prédire le prix d’un service.  
Il s’agit d’un modèle dense à plusieurs couches, adapté aux jeux de données tabulaires enrichis (numériques + embeddings).

Le modèle est conçu pour apprendre une **fonction de régression** sur les variables d’entrée (dont les embeddings de description) vers une **valeur continue de prix**.

### 🎯 4.2. Résultat attendu

- Un modèle Keras `Sequential` est initialisé avec 3 couches :
  - Deux couches cachées avec activations ReLU.
  - Une couche de sortie sans activation (régression directe).
- Une couche de régularisation `Dropout` est intégrée pour réduire le risque de surapprentissage.
- Le modèle est compilé avec :
  - L’optimiseur `adam`
  - La fonction de perte `mse`
  - L’indicateur de performance `mae`

---

### 🐍 4.3. Script de définition du modèle MLP

In [11]:
# Construction du modèle MLP (Multilayer Perceptron)

# Initialisation d'un modèle séquentiel Keras
model = Sequential([

    # Première couche cachée dense avec 128 neurones et une activation ReLU
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),

    # Couche de dropout pour limiter le surapprentissage (20% des neurones désactivés à chaque itération)
    Dropout(0.2),

    # Deuxième couche cachée dense avec 64 neurones et une activation ReLU
    Dense(64, activation='relu'),

    # Couche de sortie : une seule valeur continue (régression du prix)
    Dense(1)
])

# Compilation du modèle avec :
# - l'optimiseur 'adam' (rapide et efficace pour la majorité des cas)
# - la fonction de perte 'mse' (erreur quadratique moyenne, adaptée à la régression)
# - l'indicateur de performance 'mae' (erreur absolue moyenne, plus lisible pour l'utilisateur final)
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

---

## 🏋️‍♂️ 5. Entraînement du modèle avec EarlyStopping

### ❓ 5.1. Pourquoi cette étape maintenant ?

Après avoir préparé les données et construit notre modèle de deep learning, il est temps de lancer l’entraînement.  
Nous utilisons ici un mécanisme de **surveillance automatique** pour éviter le surapprentissage (`EarlyStopping`).

Ce mécanisme permet :
- d'interrompre l'entraînement si le modèle ne s'améliore plus sur les données de validation,
- d'éviter d’apprendre des détails trop spécifiques à l’échantillon d’entraînement (overfitting),
- de restaurer automatiquement les **meilleurs poids** enregistrés.

### 🎯 5.2. Résultat attendu

- Le modèle est entraîné sur les données normalisées `X_train_scaled`.
- Une **validation croisée interne** est effectuée à chaque époque sur 20% des données.
- Le processus s’interrompt automatiquement si aucune amélioration n’est constatée pendant 10 époques consécutives.
- L’objet `history` contient toutes les informations nécessaires à la visualisation de la courbe d’apprentissage.

---

### 🐍 5.3. Script d’entraînement avec arrêt anticipé

In [12]:
# Entraînement avec early stopping

# Création d’un callback EarlyStopping :
# - 'monitor' : indique que l'on surveille la perte sur les données de validation ('val_loss').
# - 'patience' : arrête l'entraînement si la perte ne s'améliore pas après 10 epochs consécutifs.
# - 'restore_best_weights' : restaure les poids du modèle obtenus à l'époque avec la meilleure val_loss.
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entraînement du modèle sur les données d’entraînement
# - 'validation_split' : 20% des données d'entraînement sont utilisées pour valider le modèle pendant l'entraînement.
# - 'epochs' : nombre maximum d'itérations (epochs).
# - 'batch_size' : nombre d'exemples traités avant la mise à jour des poids.
# - 'callbacks' : utilise le mécanisme d'arrêt anticipé pour éviter l’overfitting.
# - 'verbose' : 1 pour affichage détaillé de la progression dans le terminal.
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
23/23 [==============================] - 1s 14ms/step - loss: 36.9353 - mae: 4.5490 - val_loss: 34.3361 - val_mae: 4.6683
Epoch 2/100
23/23 [==============================] - 0s 5ms/step - loss: 25.0065 - mae: 3.8876 - val_loss: 35.1867 - val_mae: 4.6080
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 21.3895 - mae: 3.6078 - val_loss: 34.1707 - val_mae: 4.6207
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 18.3212 - mae: 3.3273 - val_loss: 34.8686 - val_mae: 4.5734
Epoch 5/100
23/23 [==============================] - 0s 6ms/step - loss: 16.5729 - mae: 3.1223 - val_loss: 34.3977 - val_mae: 4.5513
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 14.3335 - mae: 2.8877 - val_loss: 33.6412 - val_mae: 4.4827
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 12.6017 - mae: 2.6869 - val_loss: 34.6260 - val_mae: 4.5464
Epoch 8/100
23/23 [==============================] - 0s 6ms/step - l

---

## 🧪 6. Évaluation du modèle entraîné

### ❓ 6.1. Pourquoi cette étape maintenant ?

L’évaluation finale permet de mesurer la **performance réelle** du modèle sur des données **inédites** (non vues pendant l’entraînement).  
Cela permet de détecter :
- Un éventuel **surapprentissage** si la performance chute trop par rapport au jeu d’entraînement,
- L’efficacité globale du modèle dans un contexte d’usage réel.

### 🎯 6.2. Résultat attendu

- Le modèle retourne deux indicateurs clés :
  - **MAE** (*Mean Absolute Error*) : écart moyen absolu entre les prix réels et prédits,
  - **MSE** (*Mean Squared Error*) : utilisé comme fonction de perte pour l’entraînement.
- Ces valeurs sont imprimées dans la console pour analyse comparative.

---

### 🐍 6.3. Script d’évaluation du modèle sur le jeu de test

In [13]:
# Évaluation du modèle sur les données de test

# Évaluation finale du modèle entraîné à l'aide des données de test standardisées.
# La méthode 'evaluate' retourne deux métriques :
# - loss : ici c’est le MSE (Mean Squared Error) car le modèle a été compilé avec la perte "mse"
# - mae : Mean Absolute Error, plus lisible et moins sensible aux grandes erreurs
loss, mae = model.evaluate(X_test_scaled, y_test)

# Affichage des résultats arrondis à deux décimales
print(f"\nÉvaluation finale - MAE : {mae:.2f}, MSE : {loss:.2f}")

8/8 [==============================] - 0s 2ms/step - loss: 29.3919 - mae: 3.9796

Évaluation finale - MAE : 3.98, MSE : 29.39


---

## 💾 7. Sauvegarde du modèle Deep Learning entraîné

### ❓ 7.1. Pourquoi cette étape maintenant ?

Une fois le modèle entraîné et validé, il est crucial de **le sauvegarder** afin de :
- Réutiliser le modèle plus tard sans avoir à le réentraîner,
- L’intégrer dans une application (API, Gradio, etc.),
- Conserver une version stable du modèle pour reproductibilité ou archivage.

Le format `.h5` est un format standard de sauvegarde pour les modèles Keras.

### 🎯 7.2. Résultat attendu

- Le modèle est sauvegardé dans le fichier `models/deep/deep_model.h5`.
- Un message de confirmation s’affiche dans le terminal.

---

### 🐍 7.3. Script de sauvegarde du modèle Deep Learning

In [14]:
# 🔹 Sauvegarde du modèle Keras

# Définition du chemin de sauvegarde du modèle Deep Learning
model_path = "../models/deep/deep_model_notebook.h5"

# Sauvegarde du modèle Keras au format HDF5 (.h5)
model.save(model_path)

# Confirmation de la sauvegarde
print("Modèle sauvegardé :", model_path)

Modèle sauvegardé : ../models/deep/deep_model_notebook.h5
